In [43]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import requests
import json

In [ ]:
browser = Browser('chrome')

In [ ]:
# Visit the Mars news site
url = 'https://www.billboard.com/charts/tiktok-billboard-top-50/'
browser.visit(url)

In [ ]:
# Create a Beautiful Soup object
html = browser.html
soup = soup(html, 'html.parser')

In [ ]:
# Extract all the text elements
Top_chart_elements = soup.find_all("div", class_="o-chart-results-list-row-container")

In [ ]:
# Create an empty list to store the dictionaries
top_chart_list = []

In [ ]:
# Loop through the text elements
for article in Top_chart_elements:
    
    date = "2024-06-08"
    title = article.find("h3", class_="c-title").get_text()
    try:
        artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only u-font-size-20@tablet").get_text()
    except:
        artist = artist = article.find("span", class_="c-label a-no-trucate a-font-primary-s lrv-u-font-size-14@mobile-max u-line-height-normal@mobile-max u-letter-spacing-0021 lrv-u-display-block a-truncate-ellipsis-2line u-max-width-330 u-max-width-230@tablet-only").get_text()

    
    track_name = title.strip()
    artist_name = artist.strip()
    
    # Store in a dictionary
    article_dict = {
        "title": track_name,
        "artist": artist_name,
        "date": date
    }
    
    # Append the dictionary to the list
    top_chart_list.append(article_dict)

In [59]:
top_chart_list
top_chart_df = pd.DataFrame(top_chart_list)
top_chart_df

,title,artist,date
0,Million Dollar Baby,Tommy Richman,2024-06-08
1,Tell Ur Girlfriend,Lay Bankz,2024-06-08
2,U My Everything,Sexyy Red & Drake,2024-06-08
3,Birds Of A Feather,Billie Eilish,2024-06-08
4,Not Like Us,Kendrick Lamar,2024-06-08
5,Crazy,Stephen Kramer Glickman Featuring Marza Wilks,2024-06-08
6,Type Shit,"Future, Metro Boomin, Travis Scott & Playboi C...",2024-06-08
7,BAND4BAND,Central Cee & Lil Baby,2024-06-08
8,Si No Quieres No,Luis R Conriquez x Neton Vega,2024-06-08
9,Gata Only,FloyyMenor X Cris Mj,2024-06-08


In [20]:
browser.quit()

In [74]:
import requests

CLIENT_ID = "your_id_here"
CLIENT_SECRET = "your_secret_here"

AUTH_URL = "https://accounts.spotify.com/api/token"
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': "68eea273ba0b4954a6237107625186fe",
    'client_secret': "18597e9724e945c8a64f86a2093aaea3",
})

#Convert response to JSON
auth_response_data = auth_response.json()

#Save the access token
access_token = auth_response_data['access_token']

#Need to pass access token into header to send properly formed GET request to API server
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [75]:
artist = "Tommy Richman"
trackName = "Million Dollar Baby"

In [76]:
BASE_URL = 'https://api.spotify.com/v1/'
r = requests.get(BASE_URL + f'search?q={artist} {title}&type=artist,track&offset=0&limit=1', headers=headers)
r = r.json()


In [77]:
r

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Tommy+Richman+%0A%0A%09%0A%09%0A%09%09%0A%09%09%09%09%09Laced+Max%09%09%0A%09%0A&type=artist&offset=0&limit=1',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1WaFQSHVGZQJTbf0BdxdNo'},
    'followers': {'href': None, 'total': 426295},
    'genres': ['chill abstract hip hop'],
    'href': 'https://api.spotify.com/v1/artists/1WaFQSHVGZQJTbf0BdxdNo',
    'id': '1WaFQSHVGZQJTbf0BdxdNo',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab6761610000e5ebbb330168f192dbdc28fcbd37',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/ab67616100005174bb330168f192dbdc28fcbd37',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/ab6761610000f178bb330168f192dbdc28fcbd37',
      'width': 160}],
    'name': 'Tommy Richman',
    'popularity': 81,
    'type': 'artist',
    'uri': 'spotify:artist:1WaFQSHVGZQJTbf0BdxdNo'}],
  'limit': 1,
  '

In [78]:
track_info = r['tracks']['items'][0]['id']

In [79]:
track_info

'5iwUwADk4uQUvgMF6PlTRl'

In [80]:
BASE_URL = 'https://api.spotify.com/v1/'

URI_list = []

for i, row in top_chart_df.iterrows():
    track = row[0]
    artist = row[1]
    r = requests.get(BASE_URL + f'search?q={artist}+{track}&type=artist,track&offset=0&limit=1', headers=headers)
    info = r.json()
    URI = info['tracks']['items'][0]['id']
    URI_list.append(URI)

URI_list

/var/folders/sk/zpb0sgy10yz5529n99_gq9sw0000gn/T/ipykernel_69681/1445716027.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  track = row[0]
/var/folders/sk/zpb0sgy10yz5529n99_gq9sw0000gn/T/ipykernel_69681/1445716027.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  artist = row[1]


['5AJ9hqTS2wcFQCELCFRO7A',
 '3lMzT16MjAKKXF7pSZn13B',
 '2MjXWroB9wlTG2kqv3avfS',
 '6dOtVTDdiauQNBQEDOtlAB',
 '6AI3ezQ4o3HUoP6Dhudph3',
 '4PhTvtwhgfkn9jlgPEBbue',
 '28drn6tQo95MRvO0jQEo5C',
 '7iabz12vAuVQYyekFIWJxD',
 '1C84d9abZVKWHT2YYpoean',
 '6XjDF6nds4DE2BBbagZol6',
 '6NjWCIYu1W8xa3HIvcIhd4',
 '1aHy1ipWQRNauI0ee9uYfi',
 '6Uwi2Qk3H7fM4b4W4ExrAp',
 '2FQrifJ1N335Ljm3TjTVVf',
 '7tZpun18Dl4loDzTum5rbk',
 '4ZJ4vzLQekI0WntDbanNC7',
 '29TPjc8wxfz4XMn21O7VsZ',
 '629DixmZGHc7ILtEntuiWE',
 '2qSkIjg1o9h3YT9RAgYN75',
 '5rcnAHBclBs0OGV4rATEnR',
 '2GxrNKugF82CnoRFbQfzPf',
 '1xUddpWyEuYl5T3mduKnOJ',
 '6O3WfmAQIgnLBGVzZJVS40',
 '2tudvzsrR56uom6smgOcSf',
 '22yRHdYBLZMi7xnvBioqkP',
 '1i8dJGpKO0xQiKGCVslJqB',
 '0mIdZEa3kU6btHfzbEibz4',
 '3hFnQwSt5zAWaYpLuMSWB7',
 '7rIhp6EWLNtM8qFIQruJPT',
 '48lxT5qJF0yYyf2z4wB4xW',
 '3QaPy1KgI7nu9FJEQUgn6h',
 '2prqm9sPLj10B4Wg0wE5x9',
 '0e1KTuawmiFLiK0Lh3nNtM',
 '2V852FRIFO7mFgfiyDMum1',
 '37adYGaYaAWTGhBaOzX4Fh',
 '6sghEq6gM4Ugadwa12H7Sa',
 '5UNzhuQS8ak02kW0EXsI24',
 

In [88]:
for URI in URI_list:
    Track_URL = f'https://api.spotify.com/v1/audio-features/{URI}'
    print(Track_URL)
    track_info = requests.get(Track_URL, headers=headers)
    print(track_info)
    track_details = track_info.json()
    print(track_details)
    danceability = track_details['danceability']
    print(f'danceability: {danceability}')

https://api.spotify.com/v1/audio-features/5AJ9hqTS2wcFQCELCFRO7A
<Response [200]>
{'danceability': 0.852, 'energy': 0.697, 'key': 1, 'loudness': -5.52, 'mode': 0, 'speechiness': 0.0439, 'acousticness': 0.0973, 'instrumentalness': 0.00037, 'liveness': 0.0678, 'valence': 0.919, 'tempo': 138.029, 'type': 'audio_features', 'id': '5AJ9hqTS2wcFQCELCFRO7A', 'uri': 'spotify:track:5AJ9hqTS2wcFQCELCFRO7A', 'track_href': 'https://api.spotify.com/v1/tracks/5AJ9hqTS2wcFQCELCFRO7A', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5AJ9hqTS2wcFQCELCFRO7A', 'duration_ms': 155152, 'time_signature': 4}
danceability: 0.852
https://api.spotify.com/v1/audio-features/3lMzT16MjAKKXF7pSZn13B
<Response [200]>
{'danceability': 0.866, 'energy': 0.741, 'key': 7, 'loudness': -4.66, 'mode': 1, 'speechiness': 0.245, 'acousticness': 0.0933, 'instrumentalness': 0, 'liveness': 0.0297, 'valence': 0.614, 'tempo': 135.07, 'type': 'audio_features', 'id': '3lMzT16MjAKKXF7pSZn13B', 'uri': 'spotify:track:3lMzT16MjAK